#### Create metabolic network as `networkx` graph

In [1]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data
from pathway import Pathway

# suppres rdkit warnings
import rdkit
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')


# read data from csv
cpds = pd.read_csv('../GNN_toxic/data/raw/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('data/pairs_final_RPAIRS_pred.csv', index_col=0)
cofactors = pd.read_csv('data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)
pathway = Pathway()

# Create a Compound object for each row in the DataFrame and add it to the data
for index, row in cpds.iterrows():
    entry = row['Entry']
    name = row['Names']
    formula = row['Formula']
    mw = row['mol_weight']
    smiles = row['SMILES']
    is_cofactor = row['Entry'] in cofactors['Entry'].values
    is_toxic = row['toxic']
    is_polymer = row['polymer']

    compound = Compound(entry, name, formula, mw, smiles, is_cofactor, is_polymer, is_toxic)
    data.add_element('compound', compound)

# Create a Reaction object for each row in the DataFrame and add it to the data
for index, row in rxns.iterrows():
    entry = row['Entry']
    name = row['Names']
    compounds = row['Compound']
    enzyme = row['EC Number']

    reaction = Reaction(entry, name, compounds, enzyme)
    data.add_element('reaction', reaction)

# CREATE GRAPH
graph.create_graph(data=data, pairs=pairs)

graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

# nodes: 8063 
# edges: 12114


100%|██████████| 12114/12114 [00:26<00:00, 460.13it/s]


#### Get KEGG source and target

In [2]:
# define target compound
butanol_KEGG = 'C06142'

# define source list
src_list = ['C00084', 'C00024', 'C00074', 'C00022', 'C00497', 'C00149', \
            'C00122', 'C00042', 'C00036', 'C00091', 'C00111', 'C00158']

#### Finde metabolic pathways from src to trg

In [12]:
for src in src_list[:11]:
    pathway.initialize(src, butanol_KEGG, graph)
    pathway.get_pathway()
    pathway.print_pathway(with_reactions=True, to_bigg=True)    
    print('--------------------------------------')    

Pathway from C00084 to C06142:
['C00084', 'C00024', 'C00332', 'C01144', 'C00877', 'C00136', 'C01412', 'C06142']
[['R00228'], ['R00238', 'R09527'], ['R01975', 'R01976'], ['R03026'], ['R01171', 'R01175', 'R09738', 'R11675'], ['R01172', 'R01173'], ['R03544', 'R03545']]
['acald', 'accoa', 'aacoa', '3hbcoa', 'b2coa', 'btcoa', 'btal', '1btol']
[['ACALD'], ['ACACT1m', 'Not found'], ['HACD1', 'HBCO_nadp'], ['ECOAH1m'], ['ACOAD1', 'ACOAD1f', 'Not found', 'Not found'], ['BTCOARx', 'Not found'], ['ALCD4', 'BTS_nadph']]


['C00084', 'C00024', 'C00332', 'C03561', 'C00877', 'C00136', 'C01412', 'C06142']
[['R00228'], ['R00238', 'R09527'], ['R01977'], ['R03027'], ['R01171', 'R01175', 'R09738', 'R11675'], ['R01172', 'R01173'], ['R03544', 'R03545']]
['acald', 'accoa', 'aacoa', 'Not found', 'b2coa', 'btcoa', 'btal', 'btoh']
[['ACALD'], ['KAT1', 'Not found'], ['Not found'], ['Not found'], ['ACOAD1', 'ACOAD1f', 'Not found', 'Not found'], ['BTCOARx', 'Not found'], ['ALCD4', 'ALCD4y']]


['C00084', 'C00024',